In [84]:
from params import *

In [85]:
def read_data(sheet_name):
    data = pd.read_excel(f'{RAW_DATA_DIR}/dwelling-stock/dwelling_stock_by_tenure_by_LA.ods', 
                engine='odf',
                header=3,
                usecols=['CurrentONS code', 'Local Authority (incl. owned by other LAs)', 'Private Registered Provider', 'Other public sector²', 'Private sector (P)1', 'Total (P)1'],
                sheet_name=sheet_name)
    # Rename columns 
    data.rename(columns={'CurrentONS code': 'geography_code', 
                        'Local Authority (incl. owned by other LAs)': 'local_authority_owned', 
                        'Private Registered Provider': 'private_registered_provider', 
                        'Other public sector²': 'other_public_sector', 
                        'Private sector (P)1': 'private_sector', 
                        'Total (P)1': 'total'}, inplace=True)
    # Drop empty rows
    data = data[~data['geography_code'].isna()]
    data = data.melt(id_vars='geography_code',
                     value_vars=['local_authority_owned', 'private_registered_provider', 'other_public_sector', 'private_sector'],
                     var_name='tenure_type')
    data['variable_name'] = 'number_of_houses'
    return data

In [86]:
data = read_data('2022')
data.to_csv(f'{DATA_DIR}/dwelling-stock/dwelling-stock.csv')